# Data preparation für das Neuronale Netz

## Importieren der Libraries

In [13]:
import pandas as pd
import numpy as np
import os

## Importieren des Datensatzes sowie Aufteilung der Umsätze in eine Spalte pro Warengruppe

In [14]:
# Laden des Datensatzes
dataf = pd.read_csv("https://raw.githubusercontent.com/FabsenMc/bakery_prediction/main/0_DataPreparation/dataf.csv")

# Convert the date column to datetime (sicherheitshalber)
dataf['Datum'] = pd.to_datetime(dataf['Datum'])

# prüfen welche Spalten in Categories umgewandelt werden müssen
print(dataf.dtypes)

# Pivot the data to have one row per date and one column per category
pivot_dataf = dataf.pivot_table(index='Datum', columns='Warengruppe', values='Umsatz').reset_index()

#New created columns get the prefix 'Warengruppe_'
pivot_dataf.columns = ['Datum'] + ['Warengruppe_' + str(col) for col in pivot_dataf.columns if col != 'Datum']

# Ensure no missing values by filling them with zeros (hier vmtl wichtig für Warengruppe 6)
pivot_dataf.fillna(0, inplace=True)

# Change the type of all columns with the prefix 'Warengruppe_' to float
for col in pivot_dataf.columns:
    if 'Warengruppe_' in col:
        pivot_dataf[col] = pivot_dataf[col].astype(float)

# Anzeigen lassen ob die Pivotierung funktioniert hat und wie die Spalten heißen
print(pivot_dataf.head())
print(pivot_dataf.dtypes)



Datum                   datetime64[ns]
KielerWoche                    float64
Warengruppe                    float64
Umsatz                         float64
Bewoelkung                     float64
Temperatur                     float64
Windgeschwindigkeit            float64
Wettercode                     float64
Beschreibung                    object
FerienSH                       float64
Feiertag                       float64
Uhrzeit                         object
Heim_Auswärts                   object
Umschlag                       float64
Weihnachtsmarkt                float64
Regen                            int64
Wochentag_MDMDFSS                int64
Wochenende                       int64
Jahreszeit_FSHW                  int64
Temperatur_Kategorie            object
dtype: object
       Datum  Warengruppe_1.0  Warengruppe_2.0  Warengruppe_3.0  \
0 2013-07-01       148.828353       535.856285       201.198426   
1 2013-07-02       159.793757       546.780787       265.261254   
2 201

## Kategorielle Variablen definieren und mit dem restlichen Datensatz vereinigen

In [15]:
# Features aus dem Datensatz herausziehen (hier nur beispielhaft, muss erweitert werden), die ins neuronale Netz eingehen sollen (es scheinen alle am Ende Kategorial zu sein bei uns)
categorical_features = ['Temperatur_Kategorie', 'Bewoelkung']

# Überprüfen der einzigartigen Werte in den kategorialen Features und umwandlung in Type category falls es nicht schon vorher so war
print("Unique Values:\n",dataf[categorical_features].apply(lambda x: x.unique()))
for col in categorical_features:
    dataf[col] = dataf[col].astype('category')

# Die gewählten Features hier in dummies codieren
features = pd.get_dummies(dataf[categorical_features], drop_first=True, dtype=int)

# Dependend Varibale (Bei uns der Umsatz pro Warengruppe) mit den Features zusammenfügen
prepared_data = pd.concat([pivot_dataf, features], axis=1)

# missing values werden entfernt
prepared_data = prepared_data.dropna()

#DF nach Datum sortieren
prepared_data = prepared_data.sort_values(by='Datum')

# Display the shape of the prepared data set
print("Prepared data shape:\n", prepared_data.shape)
# Display the first few rows of the prepared data set
print("Prepared data first rows:\n", prepared_data.head())

# Define the feature columns and target columns
feature_columns = ['Temperatur_Kategorie', 'Bewoelkung']
feature_columns = prepared_data.columns[prepared_data.columns.str.startswith('Temperatur_Kategorie') | prepared_data.columns.str.startswith('Bewoelkung')]
target_columns = prepared_data.columns[prepared_data.columns.str.startswith('Warengruppe_')]

print("Feature Columns:\n", feature_columns)
print("Target Columns:\n", target_columns)

Unique Values:
 Temperatur_Kategorie                         [Niedrig, nan, Mittel, Hoch]
Bewoelkung              [8.0, 7.0, 4.0, 6.0, 3.0, 2.0, 1.0, 5.0, 0.0, ...
dtype: object
Prepared data shape:
 (1819, 17)
Prepared data first rows:
        Datum  Warengruppe_1.0  Warengruppe_2.0  Warengruppe_3.0  \
0 2013-07-01       148.828353       535.856285       201.198426   
1 2013-07-02       159.793757       546.780787       265.261254   
2 2013-07-03       111.885594       427.343259       210.260241   
3 2013-07-04       168.864941       454.859641       190.686641   
4 2013-07-05       171.280754       492.818804       181.644870   

   Warengruppe_4.0  Warengruppe_5.0  Warengruppe_6.0  \
0        65.890169       317.475875              0.0   
1        74.543917       383.628682              0.0   
2        69.262728       305.523072              0.0   
3        61.490175       308.408168              0.0   
4        86.759861       355.518770              0.0   

   Temperatur_Kategori

## Split des Datensatzes in Trainings und Validierungsdaten

In [16]:
# Hier muss vermutlich nochmnal eine Randomisierung rein (durchmischen damit nicht zufällig in einem Batch zusammenhängende Daten landen)
# vermutlich nach dem auftrennen der Datensätze in die zwei Gruppen (da geht Datum eh verloren)


# Definieren der Datumsgrenzen
train_start_date = '2013-07-01'
train_end_date = '2017-07-31'
validation_end_date = '2018-07-31'

# Split the data into training (3 years) and validation (1 year)
train_data = prepared_data[(prepared_data['Datum']>= train_start_date) & (prepared_data['Datum'] <= train_end_date)]
val_data = prepared_data[(prepared_data['Datum']> train_end_date) & (prepared_data['Datum'] <= validation_end_date)]

print(train_data.head())
print(val_data.head())

# Überprüfen der Dimensionen der Datensätze
print("Training dataset dimensions:", train_data.shape)
print("Validation dataset dimensions:", val_data.shape)

# Separating features and labels/targets
training_features = train_data[feature_columns].values
validation_features = val_data[feature_columns].values

training_labels = train_data[target_columns].values
validation_labels = val_data[target_columns].values

# Print dimensions of the features and labels
print("Training features dimensions:", training_features.shape)
print("Validation features dimensions:", validation_features.shape)
print()
print("Training labels dimensions:", training_labels.shape)
print("Validation labels dimensions:", validation_labels.shape)

       Datum  Warengruppe_1.0  Warengruppe_2.0  Warengruppe_3.0  \
0 2013-07-01       148.828353       535.856285       201.198426   
1 2013-07-02       159.793757       546.780787       265.261254   
2 2013-07-03       111.885594       427.343259       210.260241   
3 2013-07-04       168.864941       454.859641       190.686641   
4 2013-07-05       171.280754       492.818804       181.644870   

   Warengruppe_4.0  Warengruppe_5.0  Warengruppe_6.0  \
0        65.890169       317.475875              0.0   
1        74.543917       383.628682              0.0   
2        69.262728       305.523072              0.0   
3        61.490175       308.408168              0.0   
4        86.759861       355.518770              0.0   

   Temperatur_Kategorie_Mittel  Temperatur_Kategorie_Niedrig  Bewoelkung_1.0  \
0                            0                             1               0   
1                            0                             1               0   
2                   

## Data Export

In [19]:
# Create subdirectory for the pickle files -> dadurch behalten die Daten ihre Types
subdirectory = "pickle_data"
os.makedirs(subdirectory, exist_ok=True)

# Export of the prepared data to subdirectory as pickle files

training_features.to_pickle(f"{subdirectory}/training_features.pkl")
validation_features.to_pickle(f"{subdirectory}/validation_features.pkl")
training_labels.to_pickle(f"{subdirectory}/training_labels.pkl")
validation_labels.to_pickle(f"{subdirectory}/validation_labels.pkl")

AttributeError: 'numpy.ndarray' object has no attribute 'to_pickle'